# Consumo de agua CDMX

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data'
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' #####SÓLO TRAE INFO 2016
FILE_NAME = 'aborto_legal'

## Class

In [2]:
from mariachis.models import BaseClass

class AbortoLegal(BaseClass):
    def __init__(self, base_dir: str) -> None:
        super().__init__(base_dir)

    def wrangling_aborto(self, df, clean_dict, date_col='fingreso', **kwargs):
        # Sólo registros con fecha
        df = df[df[date_col].notnull()].reset_index()
        # Crear variables de fecha
        df = self.date_vars(df, date_col)

        total_cat = (
            clean_dict['vars_just_fill_na']+
            clean_dict['vars_first_word']+
            clean_dict['vars_yes_no']+
            list(clean_dict['vars_cat'].keys())+
            list(clean_dict['vars_catbin'].keys())+
            list(clean_dict['vars_fill_zero'].keys())+
            list(clean_dict['vars_fill_one'].keys())
        )
        
        total_num = clean_dict['vars_num']+clean_dict['vars_numbin']

        # Omitir acentos de variables no numéricas
        for col in total_cat:
            df[col] = df[col].map(self.clean_text)
        
        # Sólo cambiar nulos por "DESCONOCIDO"
        for col in clean_dict['vars_just_fill_na']:
            df[col] = df[col].fillna('DESCONOCIDO')

        # Obtener la primer palabra
        for col in clean_dict['vars_first_word']:
            df[col] = df[col].str.split().str[0]

        # Agrupar categorías
        for col,to_group in clean_dict['vars_cat'].items():
            df[col] = df[col].map(to_group)

        # Agrupar binarias
        for col, to_group in clean_dict['vars_catbin'].items():
            df[col] = df[col].map(to_group)

        # Binarias de sí o no
        for col in clean_dict['vars_yes_no']:
            df[col] = df[col].map({'NO':0, 'SI':1})

        # Lo que quede vacío llenar con 0
        for col, to_fill in clean_dict['vars_fill_zero'].items():
            df[col] = df[col].map(to_fill).fillna(0)

        # Lo que quede vacío llenar con 1
        for col, to_fill in clean_dict['vars_fill_one'].items():
            df[col] = df[col].map(to_fill).fillna(1)

        # Ahora, limpiar el texto de variables numéricas
        for col in total_num:
                df[col] = df[col].map(self.clean_number)

        df = df.pivot_table(**kwargs)
        df.columns = ['_'.join([x for x in col]) if not isinstance(df.columns[0],str) else col for col in df.columns]
        return df

al = AbortoLegal(BASE_DIR)
print(al)

Directorio:	/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


## Dala Wrangling

In [3]:
df = al.full_import(api=False, file_name=FILE_NAME, to_drop=None)
df.sample()

,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
70381,2017.0,JUNIO,2017-06-16,NO,UNION LIBRE,23.0,NINGUNO,SECUNDARIA,TRABAJADORA DEL HOGAR NO REMUNERADA,NaN,...,NaN,SI,NINGUNO,NaN,NaN,NaN,DFSSA017886,DFSSA017886,MEDICAMENTO,MISOPROSTOL + MIFEPRISTONA


In [4]:
from numpy import nan

# Diccionario para aplicar limpieza por tipo de variable
full_dict = {}

# Sólo cambiar nulos por "DESCONOCIDO"
full_dict['vars_just_fill_na'] = ['entidad', 'motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep']

# Obtener la primer palabra
full_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Agrupar categorías
full_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
        nan:'DESCONOCIDO'
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'OTRO',
        'NO_ESPECIFICADO':'OTRO',
        nan:'OTRO'
    }
}

# Binarias de sí o no
full_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile', 'medicamento']

# Agrupar binarias
full_dict['vars_catbin'] = {
    'desc_derechohab':{
        'NINGUNO':0,
        'IMSS':1,
        'SEGURO POPULAR':1,
        'OTRA':1,
        'ISSSTE':1
    }
}

# Lo que quede vacío llenar con 0
full_dict['vars_fill_zero'] = {
    'nivel_edu':{
        'LICENCIATURA':1,
        'MAESTRIA':1,
        'DOCTORADO':1
    },
    'parentesco':{'PAREJA':1}
}

# Lo que quede vacío llenar con 1
full_dict['vars_fill_one'] = {
    'religion':{'NINGUNA':0},
    'h_fingreso':{nan:0}
}

# Ahora, limpiar texto de variables numéricas
full_dict['vars_num'] = ['edad', 'menarca', 'fsexual', 'sememb', 'nhijos', 'gesta', 'c_num', 'p_semgest', 'p_diasgesta']
full_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Aplicar LabelEncoder para categóricas y KNN Imputer pa las numéricas
# Convertir vars_numbin a binarias

Siguientes pasos!
- diccionario para reducir variables categóricas
- h_fingreso 0 si está vacía, caso contrario -> 1
- LabelEncoder para categóricas
- KNN Imputer para numéricas
- vars_numbin convertir a binarias
- clustering añomes+municipio con vars_cat+vars_catbin escaladas y calculando ['count','min','mean','max']
- perfilamiento
- agrupar por municipio, cómo es la distribución de los clústeres previos?
- Cómo vender lo que resuelve?

In [5]:
X = al.wrangling_aborto(df, full_dict, index=['fingreso_yearmonth', 'entidad', 'alc_o_municipio'], aggfunc=['count','min','median','mean','max'], fill_value=0)
X

/Users/efraflores/Desktop/hub/dataton_cdmx/venv/lib/python3.9/site-packages/pandas/core/apply.py:577: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.

/Users/efraflores/Desktop/hub/dataton_cdmx/venv/lib/python3.9/site-packages/pandas/core/apply.py:577: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.



count_anticonceptivo  \
fingreso_yearmonth entidad             alc_o_municipio                           
2015 - 11          CIUDAD DE MEXICO    GUSTAVO A. MADERO                     1   
2016 - 01          AGUASCALIENTES      AGUASCALIENTES                        2   
                   BAJA CALIFORNIA     MEXICALI                              1   
                   BAJA CALIFORNIA SUR LOS CABOS                             1   
                   CHIAPAS             TUXTLA GUTIÉRREZ                      1   
...                                                                        ...   
2020 - 12          MORELOS             CUERNAVACA                            3   
                                       JIUTEPEC                              1   
                                       TEMIXCO                               0   
                   NUEVO LEON          MONTERREY                             1   
                   VERACRUZ            XALAPA                                1   

                                                          count_autoref  \
fingreso_yearmonth entidad             alc_o_municipio                    
2015 - 11          CIUDAD DE MEXICO    GUSTAVO A. MADERO              1   
2016 - 01          AGUASCALIENTES      AGUASCALIENTES                 0   
                   BAJA CALIFORNIA     MEXICALI                       0   
                   BAJA CALIFORNIA SUR LOS CABOS                      0   
                   CHIAPAS             TUXTLA GUTIÉRREZ               0   
...                                                                 ...   
2020 - 12          MORELOS             CUERNAVACA                     0   
                                       JIUTEPEC                       0   
                                       TEMIXCO                        0   
                   NUEVO LEON          MONTERREY                      0   
                   VERACRUZ            XALAPA                         0   

                                                          count_año  \
fingreso_yearmonth entidad             alc_o_municipio                
2015 - 11          CIUDAD DE MEXICO    GUSTAVO A. MADERO          1   
2016 - 01          AGUASCALIENTES      AGUASCALIENTES             2   
                   BAJA CALIFORNIA     MEXICALI                   1   
                   BAJA CALIFORNIA SUR LOS CABOS                  1   
                   CHIAPAS             TUXTLA GUTIÉRREZ           1   
...                                                             ...   
2020 - 12          MORELOS             CUERNAVACA                 3   
                                       JIUTEPEC                   1   
                                       TEMIXCO                    1   
                   NUEVO LEON          MONTERREY                  1   
                   VERACRUZ            XALAPA                     1   

                                                          count_c_dolor  \
fingreso_yearmonth entidad             alc_o_municipio                    
2015 - 11          CIUDAD DE MEXICO    GUSTAVO A. MADERO              1   
2016 - 01          AGUASCALIENTES      AGUASCALIENTES                 0   
                   BAJA CALIFORNIA     MEXICALI                       0   
                   BAJA CALIFORNIA SUR LOS CABOS                      0   
                   CHIAPAS             TUXTLA GUTIÉRREZ               0   
...                                                                 ...   
2020 - 12          MORELOS             CUERNAVACA                     3   
                                       JIUTEPEC                       1   
                                       TEMIXCO                        1   
                   NUEVO LEON          MONTERREY                      1   
                   VERACRUZ            XALAPA                         1   

                                                          count_c_fecha  \
fingreso_yearmonth entidad             a